**TOY EXAMPLE**


In [1]:
import numpy as np
import pandas as pd
#  Set seed for reproducibility
np.random.seed(123)

# parameters
## pesticide usage
pesticide_share = 0.5
## worm risk
worm_risk = 0.2 # pesticide = 0
worm_risk_afterpesticide = 0.01 # pesticide = 1
## pesticide direct effect
direct_pesticide = -5
worm_reduction = 0.1

# pesticide sampling
data =  np.random.choice([0,1],size = 1000 ,p=[1 - pesticide_share, pesticide_share])
# random pesticide allocation
df = pd.DataFrame(data, columns = ['pesticide'])
# default no worms
df['worms']=0
df.loc[df['pesticide']==0,'worms']= np.random.choice([0,1],size = len(df[df['pesticide']==0]),p=[1-worm_risk ,worm_risk])
df.loc[df['pesticide']==1,'worms']= np.random.choice([0,1],size = len(df[df['pesticide']==1]),p=[1 - worm_risk_afterpesticide,worm_risk_afterpesticide])
# lambda parameter for yield
df['param']=(50 + direct_pesticide * df['pesticide'])*(worm_reduction**df['worms'])
# sample apples
df['yield'] = np.random.poisson(lam=df['param'].values, size=(1,len(df['param']))).reshape((len(df['param']),1))
# drop lambda parameter
df1 = df.drop(['param'], axis=1)

**TRAIN XGB**

In [ ]:
import xgboost as xgb

X = df1.drop(columns='yield')
y = df1['yield']



train = xgb.DMatrix(X, label=y)

classifier = xgb.train(
        params={"objective": "reg:linear","eta":0.1, "max_depth" :  3 , "subsample" : 0.8, 'eval_metric': 'mae'},
        dtrain  = train,
        num_boost_round=1000,
        early_stopping_rounds=10,
        evals=[(train, "train")],
        verbose_eval=True)

**EXPLAINABILITY**

In [ ]:
!pip install dalex
!pip install shap

In [ ]:
# import shap library
import shap
# Fits the explainer
explainer = shap.Explainer(classifier, model_output= "raw")
# Calculates the SHAP values - It takes some time
shap_values = explainer( train)

shap.summary_plot(shap_values, X, plot_type="bar")


### WORMS = 0
### PESTICIDE = 0

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values.values[2, :], df1[['pesticide','worms']].iloc[2, :])

### WORMS = 0
### PESTICIDE = 1

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values.values[95, :], df1[['pesticide','worms']].iloc[95, :], link = "identity")


### WORMS = 1
### PESTICIDE = 0

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values.values[27, :], df1[['pesticide','worms']].iloc[27, :], link = "identity")

### WORMS = 1
### PESTICIDE = 1

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values.values[94, :], df1[['pesticide','worms']].iloc[94, :], link = "identity")

**DALEX ALTERNATIVE**

In [ ]:
import dalex as dx
exp = dx.Explainer(classifier, X, y)

exp.predict(X)
exp.model_parts().plot()

In [ ]:
exp.model_profile().plot()


In [ ]:
example = exp.predict_parts( new_observation = df1[['pesticide','worms']].iloc[2, :], type = "break_down")
# plot Break Down
example.plot()

## REAL EFFECT PESTICIDE

In [ ]:
def expected_yield(pesticide,worm_risk = 0.2, worm_risk_afterpesticide = 0.01, worm_reduction = 0.1, direct_pesticide = -5 ):

  if pesticide == 0:
    worms =  np.random.choice([0,1],size = 1,p=[1-worm_risk ,worm_risk])
  else:
    worms =  np.random.choice([0,1],size = 1,p=[1-worm_risk_afterpesticide ,worm_risk_afterpesticide])
  # lambda parameter for yield
  lam = (50 + direct_pesticide * pesticide) *(worm_reduction**worms)
  # sample apples
  return(np.random.poisson(lam= lam , size=1))

### simulate
simul = 1000
no_pest = []
yes_pest = []

np.random.seed(345)
for x in range(10000):
  no_pest.append(expected_yield(0))
  yes_pest.append(expected_yield(1))

print(np.mean(yes_pest) - np.mean(no_pest))